# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    

from neural_data_analysis.topic_based_neural_analysis.replicate_one_ff.one_ff_gam import one_ff_pgam_design
from neural_data_analysis.neural_analysis_tools.pgam_tools import pgam_class
from neural_data_analysis.topic_based_neural_analysis.replicate_one_ff import population_analysis_utils, one_ff_data_processing, one_ff_pipeline, parameters

import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats
import json

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_colwidth', 200)



pgam_path = '/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/external/pgam/src/'
import sys
if not pgam_path in sys.path: 
    sys.path.append(pgam_path)
    
import numpy as np
import sys
from PGAM.GAM_library import *
import PGAM.gam_data_handlers as gdh
import matplotlib.pylab as plt
import pandas as pd
from post_processing import postprocess_results
from scipy.io import savemat


# Use class

In [ ]:
prs = parameters.default_prs()
data_obj = one_ff_pipeline.OneFFSessionData(
    mat_path='all_monkey_data/one_ff_data/sessions_python.mat',
    prs=prs, 
    session_num=0,
)

covariate_names = [
    'v', 'w', 'd', 'phi',
    'r_targ', 'theta_targ',
    'eye_ver', 'eye_hor',
]

# preprocessing
data_obj.compute_covariates(covariate_names)
Y = data_obj.compute_spike_counts()
Y_smooth = data_obj.smooth_spikes()
data_obj.compute_events()




In [ ]:
# PGAM design for one unit
unit_idx = 0
sm_handler = one_ff_pgam_design.build_smooth_handler(
    data_obj=data_obj,
    unit_idx=unit_idx,
    covariate_names=covariate_names,
    tuning_covariates=covariate_names,
    use_cyclic=set(),
    order=4,
)

sm_handler.smooths_var

In [ ]:
data_obj.prs.binrange

In [ ]:
import pandas as pd
import numpy as np

binned_spikes_df = data_obj.get_binned_spikes_df()


# PGAM runner
runner = pgam_class.PGAMclass( 
    x_var=binned_spikes_df,
    bin_width=data_obj.prs.dt,
    save_dir='all_monkey_data/one_ff_data/pgam_results'
)

# attach design + trial structure
runner.sm_handler = sm_handler
runner.trial_ids = data_obj.covariate_trial_ids
runner.train_trials = runner.trial_ids % 3 != 1
runner.kernel_h_length = 100

In [ ]:
plot_var_order = [
    # egocentric / motion state
    'f_v',
    'f_w',
    'f_d',
    'f_phi',

    # target-related
    'f_r_targ',
    'f_theta_targ',
    'g_t_targ',          # target OFF / timing

    # eye position
    'f_eye_hor',
    'f_eye_ver',

    # behavior / action
    'f_move',
    'f_stop',

    # outcome
    'g_t_rew',

    # history
    'h_spike_history',
]

In [ ]:
runner.sm_handler.smooths_var

In [ ]:
# run model
# runner.run_pgam(neural_cluster_number=unit_idx)
# runner.post_processing_results()
# runner.save_results()

## load results

In [ ]:
runner.load_pgam_results(3)
runner.plot_results()

In [ ]:
runner.load_pgam_results(3)
runner.plot_results(plot_var_order=plot_var_order)

# Fraction of neurons tuned (across populations)

In [ ]:
runner.res

In [ ]:
runner.res['variable']

In [ ]:
tuning_stats = compute_tuning.calculate_tuning_fraction([data_obj.res])

In [ ]:
tuning_stats

In [ ]:
data_obj.res['reduced_pval']

# Appendix

## default bin range (from Kaushik's code)

In [ ]:

prs.binrange.v = [0 ; 200]; %cm/s
prs.binrange.w = [-90 ; 90]; %deg/s
prs.binrange.a = [-0.36 ; 0.36]; %cm/s
prs.binrange.alpha = [-0.36 ; 0.36]; %deg/s
prs.binrange.r_targ = [0 ; 400]; %cm
prs.binrange.theta_targ = [-60 ; 60]; %cm
prs.binrange.d = [0 ; 400]; %cm
prs.binrange.phi = [-90 ; 90]; %deg
prs.binrange.h1 = [-0.36 ; 0.36]; %s
prs.binrange.h2 = [-0.36 ; 0.36]; %s
prs.binrange.eye_ver = [-25 ; 0]; %deg
prs.binrange.eye_hor = [-40 ; 40]; %deg
prs.binrange.veye_vel = [-15 ; 5]; %deg
prs.binrange.heye_vel = [-30 ; 30]; %deg
prs.binrange.phase = [-pi ; pi]; %rad
prs.binrange.target_ON = [-0.24 ; 0.48]; %s
prs.binrange.target_OFF = [-0.36 ; 0.36]; %s
prs.binrange.move = [-0.36 ; 0.36]; %s
prs.binrange.stop = [-0.36 ; 0.36]; %s
prs.binrange.reward = [-0.36 ; 0.36]; %s

In [ ]:
stop!

## check pgam var range

For an event, it seems that whether it’s treated as causal or not can actually change the units (milliseconds vs seconds)… I’m speechless.

In [ ]:
for var in sm_handler.smooths_var:
    print(var)
    print('Original range:', sm_handler.smooths_dict[var]._x.min    (), sm_handler.smooths_dict[var]._x.max())
    print('Binned range:', sm_handler.smooths_dict[var].xmin, sm_handler.smooths_dict[var].xmax)
    print('')

## see one trial

In [ ]:
# indices
session_num = 0
trial_num = 0

# params
sessions = data_obj.sessions
prs = parameters.default_prs()

# session / behaviour
session = sessions[session_num]
behaviour = session.behaviour

# trials / stats
all_trials = behaviour.trials
all_stats = behaviour.stats
trial_ids = np.arange(len(all_trials))

trial = all_trials[trial_num]
stats = all_stats[trial_num]
pos_rel = stats.pos_rel

# continuous data
continuous = trial.continuous
print(continuous._fieldnames)

x = continuous.xmp
y = continuous.ymp
v = continuous.v
w = continuous.w
t = continuous.ts

# time step
prs.dt = round(np.mean(np.diff(t)), 5)

In [ ]:
trial_data = data_obj.get_trial(trial_num=0)
trial_spikes = data_obj.get_trial_spike_times(trial_num=0)



In [ ]:
import matplotlib.pyplot as plt

plt.plot(trial_data['x'], trial_data['y'], 'k-')
plt.xlabel('x (forward)')
plt.ylabel('y (lateral)')
plt.axis('equal')


## verify compute_all_covariates

In [ ]:
covariates = one_ff_data_processing.compute_all_covariates(trial, prs.dt)


In [ ]:
import numpy as np

are_close = np.allclose(pos_rel.r_targ, covariates['r_targ'], rtol=1e-5, atol=1e-8, equal_nan=True)
print(are_close)

are_close = np.allclose(pos_rel.theta_targ, covariates['theta_targ'], rtol=1e-5, atol=1e-8, equal_nan=True)
print(are_close)

In [ ]:
covariates.keys()

In [ ]:
for k, v in covariates.items():
    print(k, v.shape, np.nanmin(v), np.nanmax(v))
